In [19]:
import pinecone
from llama_index import StorageContext
from llama_index.vector_stores import PineconeVectorStore
import os

# pinecone_api_key = os.environ.get("PINECONE_API_KEY")
# pinecone_env = os.environ.get("PINECONE_ENV")

pinecone_api_key = os.environ['PINECONE_API_KEY']
pinecone_env = os.environ['PINECONE_ENV']

# init pinecone
pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)

In [20]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
langchain_module_path = os.path.abspath(os.path.join('../LangChainStudy/'))
model_config_path = os.path.abspath(os.path.join('../LangChainStudy/custom_llms/'))
sys.path.insert(1, module_path)
sys.path.insert(1, langchain_module_path)
sys.path.insert(1, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [12]:
# pinecone免费版只能有一个index，下面是第一次创建时使用
# pinecone.create_index("xdtest1", dimension=1024, metric="cosine", pod_type="p1") # pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

In [21]:
# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store = PineconeVectorStore(pinecone.Index("xdtest1"))
)

In [22]:
# Load documents and build index
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index import GPTVectorStoreIndex
documents_p = SimpleDirectoryReader('../data/mr_fujino').load_data()


In [23]:
service_context = ServiceContext.from_defaults(llm=model, embed_model=ZhipuAIEmbeddings())

In [24]:
index_p = VectorStoreIndex.from_documents(documents_p, service_context=service_context, storage_context=storage_context)

Upserted vectors: 100%|██████████| 5/5 [00:01<00:00,  2.80it/s]


In [25]:
query_engine = index_p.as_query_engine()

In [26]:
response = query_engine.query("下面鲁迅先生以第一人称‘我’写的内容，请你用中文总结一下:")

print(response)

这段文字是鲁迅以第一人称“我”叙述的一段经历。故事发生在日本，当时鲁迅在日本学习医学。文章开头提到，日本报纸批评一位名叫藤野的不逊之人，但暗地里却早已受到他的影响。接着，鲁迅回忆起一件小事，干事在黑板上写的广告中，末句是“请全数到会勿漏为要”，并在“漏”字旁边加了一个圈，鲁迅当时觉得好笑，但后来意识到这也是在讥讽他。\n\n鲁迅将这些事情告诉了藤野先生，并与一些同学一起去责问干事。最后，流言得以消除，但干事却试图收回一封匿名信。鲁迅将这封信退还给他们。\n\n接着，鲁迅描述了自己在日本学习期间受到歧视的经历。由于中国是弱国，所以中国人被认为是低能儿。每当考试成绩公布，六十分以上的中国人会被质疑是否靠作弊取得高分。鲁迅在观看一部关于日本战胜俄国的电影时，看到中国人被描绘为间谍并被枪毙，观众们欢呼雀跃，这使鲁迅深感刺耳。\n\n在第二学年结束时，鲁迅决定不再学习医学，并告诉藤野先生。虽然藤野先生脸色悲哀，但并未多说什么。为了让藤野先生安心，鲁迅告诉他想去学生物学，因为藤野先生教给他的知识对学生物学也有用。实际上，鲁迅并没有坚决要学生物学，但为了安慰藤野先生，才这样说。\n\n在离开仙台前，藤野先生请鲁迅到他家，给他一张写着“惜别”的合影，并希望鲁迅也能送他一张。这段文字展示了鲁迅在日本学习期间所遭受的歧视以及与藤野先生的真挚感情。
